In [ ]:
import pandas as pd
import os
from google.colab import files

REVIEWS_FOLDER =  '/content/drive/MyDrive/reqs&reviews/Features 17'
MAIN_DATASET_PATH = '/content/drive/MyDrive/reqs&reviews/Features Avg to Fill.csv'
OUTPUT_FILE = 'laptops_with_avg_sentiment.csv'

all_reviews = []
for filename in sorted(os.listdir(REVIEWS_FOLDER)):
    if filename.endswith('.csv'):
        filepath = os.path.join(REVIEWS_FOLDER, filename)
        try:
            df = pd.read_csv(filepath, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(filepath, encoding='ISO-8859-1')
        all_reviews.append(df)

reviews_df = pd.concat(all_reviews, ignore_index=True)

feature_cols = ['Performance', 'Build quality', 'Battery life', 'Display', 'Gaming',
                'Graphics', 'Sound', 'Fans', 'Cooling system', 'Weight', 'Price']

reviews_df[feature_cols] = reviews_df[feature_cols].replace(0, pd.NA)

avg_scores = reviews_df.groupby('id')[feature_cols].mean()

avg_scores = avg_scores.round(7)

avg_scores = avg_scores.astype('float64')

try:
    main_df = pd.read_csv(MAIN_DATASET_PATH, encoding='utf-8')
except UnicodeDecodeError:
    main_df = pd.read_csv(MAIN_DATASET_PATH, encoding='ISO-8859-1')

main_df.set_index('id', inplace=True)
for col in feature_cols:
    if col in main_df.columns:
        main_df[col] = avg_scores[col].astype('float64')
main_df.reset_index(inplace=True)

main_df[feature_cols] = main_df[feature_cols].fillna(0)

main_df.reset_index(inplace=True)

main_df.to_csv(OUTPUT_FILE, index=False, float_format='%.7f')
files.download(OUTPUT_FILE)